In [1]:
import numpy as np
import pandas as pd 
from pylab import plt
plt.style.use('seaborn')

import math
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
import torch
import torch.nn as nn

/var/folders/mt/y75db3512zv_8r60rn4pb82h0000gn/T/ipykernel_44456/2574395251.py:4: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [2]:
def mse(a,b): return (mean_squared_error(a,b)); 
def mae(a,b): return (mean_absolute_error(a,b)); 
def r2(a,b): return (r2_score(a,b)**0.5); 
def isClose(base, known, tol=0.001): return np.abs((base - known) / base) <= tol; 
def pct(val): return str(val*100)[:5]+"%"; 

def row(name,ytest, yhat, accuracies):
    return [
        name, 
        r2(ytest, yhat),mae(ytest, yhat),mse(ytest, yhat), 
        accuracies[0], accuracies[1], accuracies[2]
    ]; 

def accuracy(test,pred,tol): 
    checks = np.isclose(test,pred, rtol=tol); 
    acc = np.sum(checks)/len(checks); 
    
    return pct(acc);  

In [3]:
data = pd.read_csv('data/ashwin_data.csv'); 
scaler = MinMaxScaler(feature_range=(-1, 1))
data = data[5:]; 
data.head()

,Time (h)_mean,Aeration rate(Fg:L/h)_mean,Sugar feed rate(Fs:L/h)_mean,Acid flow rate(Fa:L/h)_mean,Base flow rate(Fb:L/h)_mean,Air head pressure(pressure:bar)_mean,Substrate concentration(S:g/L)_mean,Vessel Volume(V:L)_mean,pH(pH:pH)_mean,Temperature(T:K)_mean,...,Air head pressure(pressure:bar)_min,Substrate concentration(S:g/L)_min,Vessel Volume(V:L)_min,pH(pH:pH)_min,Temperature(T:K)_min,PAA flow(Fpaa:PAA flow (L/h))_min,Oil flow(Foil:L/hr)_min,Ammonia shots(NH3_shots:kgs)_min,Water Flow_min,Penicillin concentration(P:g/L)
5,2.0,30.0,8.0,0.0,2.258033,0.6,1.321067,58491.668,6.487767,298.05334,...,0.6,1.2891,58490.0,6.4728,298.05,5.0,22.0,0.0,0.00020,0.000995
6,2.2,30.0,8.0,0.0,8.300034,0.6,1.351500,58490.668,6.477867,298.05000,...,0.6,1.3218,58490.0,6.4728,298.05,5.0,22.0,0.0,0.51087,0.000995
7,2.4,30.0,8.0,0.0,17.645700,0.6,1.379567,58491.668,6.480233,298.05000,...,0.6,1.3523,58490.0,6.4728,298.05,5.0,22.0,0.0,1.63810,0.000994
8,2.6,30.0,8.0,0.0,26.023666,0.6,1.405133,58494.000,6.492633,298.05000,...,0.6,1.3804,58491.0,6.4764,298.05,5.0,22.0,0.0,3.24120,0.000994
9,2.8,30.0,8.0,0.0,29.447332,0.6,1.428167,58497.000,6.508266,298.05000,...,0.6,1.4060,58494.0,6.4915,298.05,5.0,22.0,0.0,5.20950,0.000993


In [4]:
look_back = 5; 
def load_data(stock, look_back):
    data_raw = stock.values # convert to numpy array
    data = []
    
    # create all possible sequences of length look_back
    for index in range(len(data_raw) - look_back): 
        data.append(data_raw[index: index + look_back])
    
    data = np.array(data); 
    test_set_size = int(np.round(0.09 * data.shape[0])); 
    train_set_size = data.shape[0] - (test_set_size); 
    
    x_train = data[:train_set_size,:-1,:]; 
    y_train = data[:train_set_size,-1,:]; 
    
    x_test = data[train_set_size:,:-1]; 
    y_test = data[train_set_size:,-1,:]; 
    
    return [x_train, y_train, x_test, y_test]

x_train, y_train, x_test, y_test = load_data(data, look_back)
print('x_train.shape = ',x_train.shape)
print('y_train.shape = ',y_train.shape)
print('x_test.shape = ',x_test.shape)
print('y_test.shape = ',y_test.shape)

x_train.shape =  (103667, 4, 57)
y_train.shape =  (103667, 57)
x_test.shape =  (10253, 4, 57)
y_test.shape =  (10253, 57)


In [5]:
# make training and test sets in torch
x_train = torch.from_numpy(x_train).type(torch.Tensor)
x_test = torch.from_numpy(x_test).type(torch.Tensor)
y_train = torch.from_numpy(y_train).type(torch.Tensor)
y_test = torch.from_numpy(y_test).type(torch.Tensor)
y_train.size(),x_train.size()

(torch.Size([103667, 57]), torch.Size([103667, 4, 57]))

In [6]:
# Build model
input_dim = 57
hidden_dim = 950
num_layers = 2 
output_dim = 1

class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        self.hidden_dim = hidden_dim; 
        self.num_layers = num_layers; 

        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Init hidden state & cell state with 0s
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()

        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        # Index hidden state of last time step
        # out.size() --> 100, 32, 100
        # out[:, -1, :] --> 100, 100 --> just want last time step hidden states! 
        out = self.fc(out[:, -1, :]) 
        # out.size() --> 100, 10
        return out
    
model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)

loss_fn = torch.nn.MSELoss()

optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
print(model)
print(len(list(model.parameters())))
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())

LSTM(
  (lstm): LSTM(57, 950, num_layers=2, batch_first=True)
  (fc): Linear(in_features=950, out_features=1, bias=True)
)
10
torch.Size([3800, 57])
torch.Size([3800, 950])
torch.Size([3800])
torch.Size([3800])
torch.Size([3800, 950])
torch.Size([3800, 950])
torch.Size([3800])
torch.Size([3800])
torch.Size([1, 950])
torch.Size([1])


In [7]:
# Train model
num_epochs = 100
hist = np.zeros(num_epochs)
# Number of steps to unroll
seq_dim =look_back-1  

for t in range(num_epochs):
    # Init hidden state # Don't do this if you want your LSTM to be stateful
    #model.hidden = model.init_hidden()
    
    # Forward pass
    y_train_pred = model(x_train); 

    loss = loss_fn(y_train_pred, y_train); 
    if t % 10 == 0 and t !=0:
        print("Epoch ", t, "MSE: ", loss.item()); 
    hist[t] = loss.item(); 

    optimiser.zero_grad(); 
    loss.backward(); 
    optimiser.step(); 

/opt/homebrew/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([103667, 57])) that is different to the input size (torch.Size([103667, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [ ]:
plt.plot(hist, label="Training loss")
plt.legend()
plt.show()
np.shape(y_train_pred)

In [ ]:
# make predictions
y_test_pred = model(x_test)

# invert predictions
y_train_pred = scaler.inverse_transform(y_train_pred.detach().numpy())
y_train = scaler.inverse_transform(y_train.detach().numpy())
y_test_pred = scaler.inverse_transform(y_test_pred.detach().numpy())
y_test = scaler.inverse_transform(y_test.detach().numpy())

# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(y_train[:,0], y_train_pred[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(y_test[:,0], y_test_pred[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

In [ ]:
# Visualising the results
figure, axes = plt.subplots(figsize=(15, 6))
axes.xaxis_date()

axes.plot(df_ibm[len(df_ibm)-len(y_test):].index, y_test, color = 'red', label = 'Real IBM Stock Price')
axes.plot(df_ibm[len(df_ibm)-len(y_test):].index, y_test_pred, color = 'blue', label = 'Predicted IBM Stock Price')
#axes.xticks(np.arange(0,394,50))
plt.title('IBM Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('IBM Stock Price')
plt.legend()
plt.savefig('ibm_pred.png')
plt.show()